## Plan
1. Rewrite in keras - we already bucketing sequences
2. Use an embedding layer to get drug code vectors.
3. Get drug code tree thingy.
4. Use rolling diagnosis to improve model

In [1]:
#%%time
#lets use the drug code
import marshal, gzip
pt_by_len = marshal.load(open('train_comprehensive_ptbylen.m', 'rb'))
pred_by_len = marshal.load(open('pred_comprehensive_pts.m', 'rb'))

def get_maxes():
    hidden_max = [-1000.0] * 5
    input_max = [-1000.0] * 12
    for b in pt_by_len:
        for hidden, inp, label in pt_by_len[b]:
            for i, v in enumerate(hidden):
                hidden_max[i] = max(hidden_max[i], v)
            for inp_i in inp:
                for i, v in enumerate(inp_i):
                    input_max[i] = max(input_max[i], v)
    print hidden_max
    print input_max
hidden_max = [1.0, 1.0, 1.0, 1.0, 2526]
input_max = [0.997, 1.0, 4039, 6880.7, 1041874.5, 11682.85, 1, 6, 60258, 3181, 2869, 12] #must add 1!!!


In [2]:
import random
from sklearn.model_selection import train_test_split
import numpy as np

random.seed(42)
np.random.seed(42)

def shuffle(length_arr):
    train_pt_by_len = {}
    test_pt_by_len = {}
    for length in length_arr:
        if len(length_arr[length]) >= 50:
            train_pt_by_len[length], test_pt_by_len[length] = train_test_split(pt_by_len[length], test_size=0.1, random_state=42)
    
    return train_pt_by_len, test_pt_by_len

train_pt_by_len, test_pt_by_len = shuffle(pt_by_len)
bucketlengths = {}
total = 0
for b in train_pt_by_len:
    bucketlengths[len(train_pt_by_len[b][0][1])] = len(train_pt_by_len[b])
    total += len(train_pt_by_len[b])  

def prepare_input(batch):
    batch = zip(*batch)
    
    hidden = np.array(batch[0])
    hidden = np.delete(hidden, (1,2,4),1) # something sex sex age code
    hidden[:,1][hidden[:,1] == 1.0] = 0.5577 
    
    inp = np.array([l[1:] for l in batch[1]])
    inp = np.delete(inp ,range(2,10), 2)
    
    
    labels = np.array(batch[2])
    return hidden, inp, labels
    

def random_batch(batch_size=1):
    chunk_len = np.random.choice(bucketlengths.keys(), 1, p=np.array(bucketlengths.values())/float(sum(bucketlengths.values())))[0]
    bucket = np.array(train_pt_by_len[chunk_len])    
    hidden, inp, labels = prepare_input(bucket[np.random.choice(bucket.shape[0], batch_size)])
    
    return hidden, inp, labels

test_bucketlengths = {}
test_total = 0
for b in test_pt_by_len:
    test_bucketlengths[len(test_pt_by_len[b][0][1])] = len(test_pt_by_len[b])
    test_total += len(test_pt_by_len[b])

def random_test_batch(batch_size=1):
    chunk_len = np.random.choice(test_bucketlengths.keys(), 1, p=np.array(test_bucketlengths.values())/float(sum(test_bucketlengths.values())))[0]
    bucket = np.array(test_pt_by_len[chunk_len])    
    hidden, inp, labels = prepare_input(bucket[np.random.choice(bucket.shape[0], batch_size)])
    
    return hidden, inp, labels

print total, test_total

245794 27453


In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from tqdm import tqdm, tqdm_notebook
import numpy as np
from sklearn.metrics import roc_auc_score

def torchify_batch(hidden, inp, target=None):
    drugs = inp[:, :, -2]
    dxs = inp[:, :, -1]
    inp = inp[:, :, :-2]
    hidden = np.repeat(hidden[:,np.newaxis, :], repeats=inp.shape[1], axis=1)
    
    inp = Variable(torch.from_numpy(inp), requires_grad=False)
    drugs = Variable(torch.from_numpy(drugs), requires_grad=False).long()
    dxs = Variable(torch.from_numpy(dxs), requires_grad=False).long()
    hidden = Variable(torch.from_numpy(hidden), requires_grad=False)
    if target is not None:
        target = Variable(torch.from_numpy(target), requires_grad=False).long()
    
    return hidden, inp, drugs, dxs, target

def train(chkpoint, epochs, model, criterion, optimizer, train_losses, test_losses, test_aucs, cur_epoch=0, **kwargs):
    total = 245794
    for epoch in xrange(cur_epoch+1, epochs+1):
        print('---Epoch {}---'.format(epoch))
        num_cycles = total / 40
        #num_cycles = 5
        for cycle in tqdm_notebook(xrange(num_cycles)):
            hidden, inp, target = random_batch(40)
            hidden, inp, drugs, dxs, target = torchify_batch(hidden, inp, target)
            target.cuda()
            optimizer.zero_grad()
            out = model(hidden.cuda(), inp.cuda(), drugs.cuda(), dxs.cuda())
            loss = criterion(out, target.cuda())
            loss.backward()
            optimizer.step()
        loss_var = loss.data.cpu().numpy()[0]

        total_test_auc = 0
        total_test_loss = 0

        all_true = np.zeros((4000, 2))
        all_pred = np.zeros((4000, 2))
        
        batch_aucs = []
        bootstrap = [validate(model) for _ in range(10)]
        aucs = np.array([x[1] for x in bootstrap])
        tests = np.array([x[0] for x in bootstrap])

        test_loss, test_loss_std = np.mean(tests), np.std(tests)
        test_auc, test_auc_std = np.mean(aucs), np.std(aucs)

        train_losses.append(loss_var)
        test_losses.append(test_loss)
        test_aucs.append(test_auc)

        torch.save({
                'cur_epoch': epoch,
                'train_losses': train_losses,
                'test_losses': test_losses,
                'model': model,
                'criterion': criterion,
                'optimizer': optimizer,
                'test_aucs': test_aucs,
            }, chkpoint)

        print('Train Loss: {}'.format(loss_var)) # pass arguments to affect saving / resume behaviour
        print('Test Loss: {} Std: {}'.format(test_loss, test_loss_std))
        print('Test AUC: {} Std: {}'.format(test_auc, test_auc_std))
        
def validate(model):
    total_test_loss = 0
    all_true = np.zeros((4000, 2))
    all_pred = np.zeros((4000, 2))
    for i in xrange(100):
        hidden, inp, target = random_test_batch(40)
        target2 = target[:]
        hidden, inp, drugs, dxs, target = torchify_batch(hidden, inp, target)
        out = model(hidden.cuda(), inp.cuda(), drugs.cuda(), dxs.cuda())

        target.cuda()

        loss = criterion(out, target.cuda())
        out_np = out.data.cpu().numpy()
        for j in xrange(40):
            all_true[i*40 + j, int(target2[j])] = 1
            all_pred[i*40+j] = out_np[j]
        total_test_loss += loss.data.cpu().numpy()[0]

    test_loss = total_test_loss/100.0
    test_auc = roc_auc_score(all_true, all_pred)
    return test_loss, test_auc
    

def generate_predictions_csv(model, filename):
    predictions = []
    for b in tqdm_notebook(pred_by_len):
        hidden, inp, pt_id = prepare_input(pred_by_len[b])
        hidden, inp, drugs, dxs, _ = torchify_batch(hidden, inp)
        out = model(hidden.cuda(), inp.cuda(), drugs.cuda(), dxs.cuda()).data.cpu().numpy()
        predictions.extend(zip(pt_id, out[:,1]))

    predictions.sort()
    with open(filename, 'w') as f:
        f.write('Patient_ID,Diabetes\n')
        for a, b in predictions:
            f.write('{},{}\n'.format(a,b))

289/|/  5%|| 289/6144 [00:30<10:08,  9.62it/s]

## 3 * 32 GRU Model

In [7]:
class Model32(nn.Module):
    def __init__(self):
        super(Model32, self).__init__()
        self.drug_embedding = nn.Embedding(2870, 30)#changed from 30
        self.dx_embedding = nn.Embedding(12, 5) #changed from 5 to 7
        self.gru1 = nn.GRU(39, 32, 3, batch_first=True)
        self.dense1 = nn.Linear(32, 2)
        self.softmax = nn.Softmax()
    
    def forward(self, hidden, inp, drugs, dxs):
        drugs_e = self.drug_embedding(drugs)
        dxs_e = self.dx_embedding(dxs)
        final_input = torch.cat((hidden, inp, drugs_e, dxs_e),dim=2)
    
        h_1 = Variable(torch.zeros(3,inp.size(0), 32).double(), requires_grad=False).cuda()
        output, h_2 = self.gru1(final_input, h_1)

        x = self.dense1(h_2[-1])
        x = self.softmax(x)
        
        return x

## SGD Optimizer

In [9]:
np.random.seed(42)
torch.manual_seed(42)

model = Model32()
model.double()
model.cuda()
criterion = nn.CrossEntropyLoss().cuda()
train_losses = []
test_losses = []
test_aucs = []

train(chkpoint='model_sgd_3_32_lr_001.tar', 
      epochs=30, 
      model=model, 
      optimizer=torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9), 
      criterion=criterion, 
      test_losses=test_losses,
     train_losses=train_losses,
     test_aucs=test_aucs)

---Epoch 1---



Train Loss: 0.363613716922
Test Loss: 0.354467478091 Test AUC: 0.952723765786
Batch AUC Mean: 0.930479170198 Std: 0.126598077124
---Epoch 2---


/home/ubuntu/.local/lib/python2.7/site-packages/torch/serialization.py:147: UserWarning: Couldn't retrieve source code for container of type Model32. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "



Train Loss: 0.340076032525
Test Loss: 0.355838868997 Test AUC: 0.947812186742
Batch AUC Mean: 0.927983755135 Std: 0.139277207597
---Epoch 3---



Train Loss: 0.338502390073
Test Loss: 0.353232610323 Test AUC: 0.965355643049
Batch AUC Mean: 0.93217706823 Std: 0.141842254244
---Epoch 4---



Train Loss: 0.392691141954
Test Loss: 0.348199865391 Test AUC: 0.972527187585
Batch AUC Mean: 0.957450979356 Std: 0.10683801264
---Epoch 5---


KeyboardInterrupt: 

In [14]:
data_dict = torch.load('model_sgd_3_32_lr_001_run3.tar')
data_dict['optimizer'] = torch.optim.SGD(model.parameters(), lr=0.001, momentum=1.3, nesterov=True)
train(chkpoint='model_sgd_3_32_lr_001_run4.tar', epochs=30, **data_dict)

---Epoch 13---


600/|/ 10%|| 600/6144 [01:00<09:15,  9.97it/s]
Train Loss: 0.338426376404
Test Loss: 0.351374972434 Std: 0.00398626830214
Test AUC: 0.958150156289 Std: 0.00631060034699
---Epoch 14---



Train Loss: 0.31328814682
Test Loss: 0.352768311726 Std: 0.00488950937157
Test AUC: 0.961349765474 Std: 0.00526094230132
---Epoch 15---



Train Loss: 0.370121532312
Test Loss: 0.35284096147 Std: 0.00414302503814
Test AUC: 0.955419710623 Std: 0.00566349815491
---Epoch 16---



Train Loss: 0.388284919764
Test Loss: 0.349717204439 Std: 0.00252542585631
Test AUC: 0.958851174491 Std: 0.00535429891952
---Epoch 17---


4969/|/ 81%|| 4969/6144 [07:37<01:48, 10.86it/s]
Train Loss: 0.31330393507
Test Loss: 0.350846381482 Std: 0.00405051725321
Test AUC: 0.957735492461 Std: 0.00903707978477
---Epoch 18---



Train Loss: 0.382957154929
Test Loss: 0.352750980873 Std: 0.00431656454256
Test AUC: 0.957865994948 Std: 0.00681495489163
---Epoch 19---


KeyboardInterrupt: 

In [ ]:
class Model1(nn.Module):
    def __init__(self):
        super(Model1, self).__init__()
        self.drug_embedding = nn.Embedding(2870, 30)#changed from 30
        self.dx_embedding = nn.Embedding(12, 5) #changed from 5 to 7
        self.gru1 = nn.GRU(39, 32, 3, batch_first=True)
        self.dense1 = nn.Linear(32, 2)
        self.softmax = nn.Softmax()
    
    def forward(self, hidden, inp, drugs, dxs):
        drugs_e = self.drug_embedding(drugs)
        dxs_e = self.dx_embedding(dxs)
        final_input = torch.cat((hidden, inp, drugs_e, dxs_e),dim=2)
    
        h_1 = Variable(torch.zeros(3,inp.size(0), 32).double(), requires_grad=False).cuda()
        output, h_2 = self.gru1(final_input, h_1)

        x = self.dense1(h_2[-1])
        x = self.softmax(x)
        
        return x

epochs = 30

np.random.seed(42)
torch.manual_seed(42)

model = Model1()
model.double()
model.cuda()
criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
train_losses = []
test_losses = []
test_aucs = []

train(chkpoint='model_3_32_lr_001.tar', 
      epochs=epochs, 
      model=model, 
      optimizer=optimizer, 
      criterion=criterion, 
      test_losses=test_losses,
     train_losses=train_losses,
     test_aucs=test_aucs)

---Epoch 1---


In [ ]:
torch.load()

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from tqdm import tqdm, tqdm_notebook
import numpy as np
from sklearn.metrics import roc_auc_score

class Model1(nn.Module):
    def __init__(self):
        super(Model1, self).__init__()
        self.drug_embedding = nn.Embedding(2870, 300)#changed from 30
        self.dx_embedding = nn.Embedding(12, 5) #changed from 5 to 7
        self.gru1 = nn.GRU(309, 100, 3, batch_first=True)
        self.dense1 = nn.Linear(100, 50)
        self.dense2 = nn.Linear(50, 2)
        self.softmax = nn.Softmax()
    
    def forward(self, hidden, inp, drugs, dxs):
        drugs_e = self.drug_embedding(drugs)
        #print drugs_e.size()
        dxs_e = self.dx_embedding(dxs)
        #print dxs_e.size()
        final_input = torch.cat((hidden, inp, drugs_e, dxs_e),dim=2)
        #print final_input.size()
        h_1 = Variable(torch.zeros(3,inp.size(0), 100).double(), requires_grad=False).cuda()
        output, h_2 = self.gru1(final_input, h_1)

        x = self.dense1(h_2[-1])
        x = self.dense2(x)
        x = self.softmax(x)
        
        return x

epochs = 30

np.random.seed(42)
torch.manual_seed(42)

model = Model1()
model.double()
model.cuda()
criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
train_losses = []
test_losses = []
test_aucs = []

for epoch in xrange(1, epochs+1):
    print('---Epoch {}---'.format(epoch))
    num_cycles = total / 40
    #num_cycles = 5
    for cycle in tqdm_notebook(xrange(num_cycles)):
        hidden, inp, target = random_batch(40)
        hidden, inp, drugs, dxs, target = torchify_batch(hidden, inp, target)
        target.cuda()
        optimizer.zero_grad()
        out = model(hidden.cuda(), inp.cuda(), drugs.cuda(), dxs.cuda())
        loss = criterion(out, target.cuda())
        loss.backward()
        optimizer.step()
    loss_var = loss.data.cpu().numpy()[0]
    
    total_test_auc = 0
    total_test_loss = 0
    
    all_true = np.zeros((4000, 2))
    all_pred = np.zeros((4000, 2))
    for i in xrange(100):
        hidden, inp, target = random_test_batch(40)
        target2 = target[:]
        hidden, inp, drugs, dxs, target = torchify_batch(hidden, inp, target)
        out = model(hidden.cuda(), inp.cuda(), drugs.cuda(), dxs.cuda())
        
        target.cuda()
        
        loss = criterion(out, target.cuda())
        out_np = out.data.cpu().numpy()
        for j in xrange(40):
            all_true[i*40 + j, int(target2[j])] = 1
            all_pred[i*40+j] = out_np[j]
            
        
        total_test_loss += loss.data.cpu().numpy()[0]
        
    test_loss = total_test_loss/100.0
    test_auc = roc_auc_score(all_true, all_pred)
    
    train_losses.append(loss_var)
    test_losses.append(test_loss)
    test_aucs.append(test_auc)
    
    torch.save({
            'epoch': epoch,
            'train_losses': train_losses,
            'test_losses': test_losses,
            'state_dict': model.state_dict(),
            'test_aucs': test_aucs,
        }, 'checkpoint-model3_3_100_lr001_5_300.tar' )
    
    print('Train Loss: {}'.format(loss_var)) # pass arguments to affect saving / resume behaviour
    print('Test Loss: {} Test AUC: {}'.format(test_loss, test_auc))

---Epoch 1---



Train Loss: 0.388261687518
Test Loss: 0.374261592109 Test AUC: 0.885657979334
---Epoch 2---



Train Loss: 0.338261687518
Test Loss: 0.368512099234 Test AUC: 0.924834995698
---Epoch 3---



Train Loss: 0.413261687518
Test Loss: 0.368423856094 Test AUC: 0.942139390275
---Epoch 4---



Train Loss: 0.388265944552
Test Loss: 0.34894120233 Test AUC: 0.961716784705
---Epoch 5---



Train Loss: 0.338304045674
Test Loss: 0.353904318868 Test AUC: 0.955486463287
---Epoch 6---



Train Loss: 0.338265305098
Test Loss: 0.357419845081 Test AUC: 0.956608255739
---Epoch 7---



Train Loss: 0.388310698889
Test Loss: 0.355266816977 Test AUC: 0.953920564148
---Epoch 8---


## Rerun this one - looks pretty good

In [27]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from tqdm import tqdm, tqdm_notebook
import numpy as np
from sklearn.metrics import roc_auc_score

class Model1(nn.Module):
    def __init__(self):
        super(Model1, self).__init__()
        self.drug_embedding = nn.Embedding(2870, 300)#changed from 30
        self.dx_embedding = nn.Embedding(12, 5) #changed from 5 to 7
        self.gru1 = nn.GRU(309, 100, 3, batch_first=True)
        self.dense = nn.Linear(100, 2)
        self.softmax = nn.Softmax()
    
    def forward(self, hidden, inp, drugs, dxs):
        drugs_e = self.drug_embedding(drugs)
        #print drugs_e.size()
        dxs_e = self.dx_embedding(dxs)
        #print dxs_e.size()
        final_input = torch.cat((hidden, inp, drugs_e, dxs_e),dim=2)
        #print final_input.size()
        h_1 = Variable(torch.zeros(3,inp.size(0), 100).double(), requires_grad=False).cuda()
        output, h_2 = self.gru1(final_input, h_1)

        x = self.dense(h_2[-1])
        x = self.softmax(x)
        
        return x
    
def torchify_batch(hidden, inp, target):
    drugs = inp[:, :, -2]
    dxs = inp[:, :, -1]
    inp = inp[:, :, :-2]
    hidden = np.repeat(hidden[:,np.newaxis, :], repeats=inp.shape[1], axis=1)
    
    inp = Variable(torch.from_numpy(inp), requires_grad=False)
    drugs = Variable(torch.from_numpy(drugs), requires_grad=False).long()
    dxs = Variable(torch.from_numpy(dxs), requires_grad=False).long()
    hidden = Variable(torch.from_numpy(hidden), requires_grad=False)
    target = Variable(torch.from_numpy(target), requires_grad=False).long()
    
    return hidden, inp, drugs, dxs, target
    

epochs = 30

np.random.seed(42)
torch.manual_seed(42)

model = Model1()
model.double()
model.cuda()
criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
train_losses = []
test_losses = []
test_aucs = []

for epoch in xrange(1, epochs+1):
    print('---Epoch {}---'.format(epoch))
    num_cycles = total / 40
    #num_cycles = 5
    for cycle in tqdm_notebook(xrange(num_cycles)):
        hidden, inp, target = random_batch(40)
        hidden, inp, drugs, dxs, target = torchify_batch(hidden, inp, target)
        target.cuda()
        optimizer.zero_grad()
        out = model(hidden.cuda(), inp.cuda(), drugs.cuda(), dxs.cuda())
        loss = criterion(out, target.cuda())
        loss.backward()
        optimizer.step()
    loss_var = loss.data.cpu().numpy()[0]
    
    total_test_auc = 0
    total_test_loss = 0
    
    all_true = np.zeros((4000, 2))
    all_pred = np.zeros((4000, 2))
    for i in xrange(100):
        hidden, inp, target = random_test_batch(40)
        target2 = target[:]
        hidden, inp, drugs, dxs, target = torchify_batch(hidden, inp, target)
        out = model(hidden.cuda(), inp.cuda(), drugs.cuda(), dxs.cuda())
        
        target.cuda()
        
        loss = criterion(out, target.cuda())
        out_np = out.data.cpu().numpy()
        for j in xrange(40):
            all_true[i*40 + j, int(target2[j])] = 1
            all_pred[i*40+j] = out_np[j]
            
        
        total_test_loss += loss.data.cpu().numpy()[0]
        
    test_loss = total_test_loss/100.0
    test_auc = roc_auc_score(all_true, all_pred)
    
    train_losses.append(loss_var)
    test_losses.append(test_loss)
    test_aucs.append(test_auc)
    
    torch.save({
            'epoch': epoch,
            'train_losses': train_losses,
            'test_losses': test_losses,
            'state_dict': model.state_dict(),
            'test_aucs': test_aucs,
        }, 'checkpoint-model2_3_100_lr001_5_300.tar' )
    
    print('Train Loss: {}'.format(loss_var)) # pass arguments to affect saving / resume behaviour
    print('Test Loss: {} Test AUC: {}'.format(test_loss, test_auc))

---Epoch 1---


Train Loss: 0.388260859454
Test Loss: 0.365261738681 Test AUC: 0.945254649828
---Epoch 2---



Train Loss: 0.338263145251
Test Loss: 0.356956505921 Test AUC: 0.930382097455
---Epoch 3---



Train Loss: 0.338262501476
Test Loss: 0.357742293018 Test AUC: 0.958542924634
---Epoch 4---



Train Loss: 0.388261743708
Test Loss: 0.350712309153 Test AUC: 0.961909516622
---Epoch 5---


KeyboardInterrupt: 

---Epoch 1---


[[[ 1.      0.5577]
  [ 1.      0.5577]
  [ 1.      0.5577]
  ..., 
  [ 1.      0.5577]
  [ 1.      0.5577]
  [ 1.      0.5577]]

 [[ 0.      0.5577]
  [ 0.      0.5577]
  [ 0.      0.5577]
  ..., 
  [ 0.      0.5577]
  [ 0.      0.5577]
  [ 0.      0.5577]]

 [[ 0.      0.5577]
  [ 0.      0.5577]
  [ 0.      0.5577]
  ..., 
  [ 0.      0.5577]
  [ 0.      0.5577]
  [ 0.      0.5577]]

 ..., 
 [[ 1.      0.81  ]
  [ 1.      0.81  ]
  [ 1.      0.81  ]
  ..., 
  [ 1.      0.81  ]
  [ 1.      0.81  ]
  [ 1.      0.81  ]]

 [[ 0.      0.5577]
  [ 0.      0.5577]
  [ 0.      0.5577]
  ..., 
  [ 0.      0.5577]
  [ 0.      0.5577]
  [ 0.      0.5577]]

 [[ 1.      0.5577]
  [ 1.      0.5577]
  [ 1.      0.5577]
  ..., 
  [ 1.      0.5577]
  [ 1.      0.5577]
  [ 1.      0.5577]]]



TypeError: forward() takes exactly 5 arguments (2 given)

In [128]:
hidden, inp, labels = random_batch()
inp.shape

(1, 77, 2885)

In [97]:
start = np.array([1, 1, 3])
np.concatenate([start[:2], np_onehot(start[2], 5)], axis=0)

array([ 1.,  1.,  0.,  0.,  0.,  1.,  0.])

In [88]:
np_onehot(1.0, 4)

array([ 0.,  1.,  0.,  0.])

In [8]:
transaction_header = ['datelog', 'qtylog', 'timediff', 'ptprice', 'wsprice', 
                      'grprice', 'deferred', 'source', 'pres', 'code', 'drug', 'dx']
zip(transaction_header, pt_by_len[2][0][1][1])

[('datelog', 0.004),
 ('qtylog', 0.235),
 ('timediff', 119),
 ('ptprice', 33.9),
 ('wsprice', 19.56),
 ('grprice', 0.0),
 ('deferred', 0),
 ('source', 2),
 ('pres', 7491),
 ('code', 285),
 ('drug', 842),
 ('dx', 6)]

Format of pt_by_len : 
pt_by_len = {1: [pt1, pt2, pt3], 2: [pt1, pt2, pt3], 3:[]}
Dictionary organised by num. of transactions as the key

pt1 = [demographic_data, transaction_list, label]

e.g. 
demographic_data = [0.0, 1.0, 0.0, 1.0, 1682]
transaction_list = [[0.0, 0.0, 3181, 2869, 11], [0.119, 0.993, 84, 34.2, 33.77, 10.49, 0, 2, 58477, 2087, 2572, 11]] 
(For historical reasons you must delete the first entry in the transaction_list... )
label = 0.0

0.119, 0.993, 84, 34.2, 33.77, 10.49, 0, 2, 58477, 2087, 2572, 11 = datelog,qtylog,timediff,ptprice, wsprice,grprice,deferred,sources[source],prescribers[pres],codes[code],drugs[drug+','+dose],dxs[dx]

[1.0, 1.0, 1.0, 1.0, 2526]
[0.997, 1.0, 4039, 6880.7, 1041874.5, 11682.85, 1, 6, 60258, 3181, 2868, 11]

In [17]:
train_pt_by_len[4]

[[[1.0, 0.0, 0.0, 1.0, 1135],
  [[0.0, 0.0, 3181, 2869, 11],
   [0.023, 0.265, 196, 23.0, 20.71, 0.0, 0, 2, 25985, 1287, 2524, 6],
   [0.023, 0.265, 21, 34.2, 26.18, 2.33, 0, 2, 46772, 2606, 2788, 11],
   [0.978, 0.708, 91, 16.49, 8.56, 0.0, 0, 2, 8109, 1308, 1824, 6]],
  0.0],
 [[0.0, 1.0, 0.0, 1.0, 376],
  [[0.0, 0.0, 3181, 2869, 11],
   [0.173, 0.265, 133, 5.6, 66.03, 73.45, 0, 2, 2555, 2460, 2296, 8],
   [0.844, 0.265, 42, 37.99, 51.76, 0.0, 0, 2, 35287, 2429, 97, 10],
   [0.99, 0.5, 308, 37.99, 51.76, 0.0, 0, 2, 35287, 2429, 97, 10]],
  0.0],
 [[0.0, 1.0, 0.0, 1.0, 1555],
  [[0.0, 0.0, 3181, 2869, 11],
   [0.038, 0.5, 77, 5.6, 18.93, 22.59, 0, 2, 52377, 1500, 1325, 11],
   [0.058, 0.708, 112, 5.6, 18.93, 22.59, 0, 2, 52377, 1500, 1325, 11],
   [0.974, 0.0, 112, 0.0, 6.42, 16.85, 0, 6, 40076, 3089, 491, 11]],
  0.0],
 [[1.0, 0.0, 0.0, 1.0, 173],
  [[0.0, 0.0, 3181, 2869, 11],
   [0.092, 0.235, 21, 34.2, 34.79, 11.51, 0, 2, 35270, 2650, 129, 11],
   [0.092, 0.453, 56, 15.0, 13.65, 0

In [ ]:
p

In [1]:
import random
random.seed(42)
test_pt_by_len = {}
for length in pt_by_len:
    random.shuffle(pt_by_len[length])
    test_pt_by_len[length] = pt_by_len[length][int(len(pt_by_len[length])*0.8):]
    pt_by_len[length] = pt_by_len[length][:int(len(pt_by_len[length])*0.8)]
    
### Example function to load a random batch of data
bucketlengths = {}
total = 0
for b in pt_by_len:
    if len(pt_by_len[b]) > 40:
        bucketlengths[len(pt_by_len[b][0][1])] = len(pt_by_len[b])
        total += len(pt_by_len[b])
import numpy
print pt_by_len[2][0]
def random_batch(batch_size=1):
    chunk_len = numpy.random.choice(bucketlengths.keys(), 1, p=numpy.array(bucketlengths.values())/float(sum(bucketlengths.values())))[0]
    bucket = numpy.array(pt_by_len[chunk_len])    
    hidden, inp, labels = zip(*bucket[numpy.random.choice(bucket.shape[0], batch_size)])
    
    #hidden, inp, labels = numpy.array(hidden), numpy.array(
    #                                                    [[[0.0,0.0,0.0,0.0,0.0,
    #                                                        0.0,0.0,len(sources)-1,len(prescribers)-1,len(codes)-1,len(drugs)-1,len(dxs)-1]]+l[1:] for l in inp]), numpy.array(labels)
    hidden, inp, labels = numpy.array(hidden), numpy.array([l[1:] for l in inp]), numpy.array(labels)
    
     
    inp = numpy.delete(inp ,range(2,9), 2)
    hidden = numpy.delete(hidden, (1,2),1)
    hidden[:,1][hidden[:,1] == 1.0] = 0.5577
    return hidden, inp, labels

NameError: name 'pt_by_len' is not defined